# Object Detection Models

## Introduction

For this project, the aim would be to go through the techniques that are used for object detection in a scene or an image. The techniques that are explored here are the __You Only Look Once (YOLO)__ and __Regions with Convolutional Neural Networks (R-CNN)__. 

The process of detecting ojects in an image or video stream coupled with their bounding boxes is what object detection. Object detection is also called object locatlisation. A bounding box is a small rectangle that surrounds the object in question/interest. Here, the input for the algorithm is usually an image and the output would be a list of bounding boces and the object classes/labels. For each of the bounding boxes, the model should be able to output the corresponding predicted class/label and its confidence that the guess it correct. 

Object detection in general are widely used in industry. For example, these models can be used in the following:
1. Self driving car - for perceiving vehicles and pedestrians.
2. Content moderation - to locate forbidden objects in the scene and its respectiv size.
3. Healthcare - detecting tumors or dangerous unwanted tissues from radiographs.
4. Manufacturing - used in assembly robots of the manufacturing chain to put together or repair products.
5. Security - to detect threats, threspasses, or count people.
6. Wildlife Conservation - to monitor the population of animals.

## Breakdown of this Notebook:
- History of the object detection techniques.
- The main approaches in object detection.
- Implementing the YOLO Architecture for fast object detection task.
- Improving upon YOLO with the Faster R-CNN architecture.
- Utilising the Faster R-CNN with the TensorFlow Object Detection API.

## Dataset:



## 




### Import the required libraries:

In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import timeit
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2

In [2]:
import os
from IPython.display import display, Image
import matplotlib.pyplot as plt

# %matplotlib inline

# Set up the working directory for the images:
image_folderName = 'Description Images'
image_path = os.path.abspath(image_folderName) + '/'

In [3]:
# Set the random set seed number: for reproducibility.
Seed_nb = 42

# Set to run or not run the code block: for code examples only. (0 = run code, and 1 = dont run code)
dont_run = 0

### GPU Information:

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
devices = sess.list_devices()
devices

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 1474424523630474660),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 6586313605, 13280329909406021941)]

### Use RTX_GPU Tensor Cores for faster compute: FOR TENSORFLOW ONLY

Automatic Mixed Precision Training in TF. Requires NVIDIA DOCKER of TensorFlow.

Sources:
- https://developer.nvidia.com/automatic-mixed-precision
- https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html#framework

When enabled, automatic mixed precision will do two things:

- Insert the appropriate cast operations into your TensorFlow graph to use float16 execution and storage where appropriate(this enables the use of Tensor Cores along with memory storage and bandwidth savings). 
- Turn on automatic loss scaling inside the training Optimizer object.

In [5]:
# os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

EXAMPLE CODE: 

In [6]:
# # Graph-based example:
# opt = tf.train.AdamOptimizer()
# opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
# train_op = opt.miminize(loss)

# # Keras-based example:
# opt = tf.keras.optimizers.Adam()
# opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
# model.compile(loss=loss, optimizer=opt)
# model.fit(...)

### Use RTX_GPU Tensor Cores for faster compute: FOR KERAS API

Source:
- https://www.tensorflow.org/guide/keras/mixed_precision
- https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/experimental/Policy

In [7]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [8]:
# Set for MIXED PRECISION:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


## 1 - History of Object Detection:

In the past, classical computer vision techniques for object detection uses image descriptors, this is where to detect an object like a bike would require several images of this object. The term descriptors refers to the bike object that would be extracted from the image and that these descriptors represents different parts of the bike. As the algorithm looks for the object (bike), it will try to find the descriptors in the target images. 

The most common technique at the time was the usage of the floating window. It is where small rectangular areas of the images were examined one by one, where the part that matches the descriptors the most would be considered to contain the object of interest. This technique have a few advantage at the time, where it was robust to rotation and colour changes in the images, it also does not require a lot training data samples and overall, it worked for most objects. The drawback was that the level of accuracy was not good enough. Soon Neural Networks outpaced this tradiational technique.

Modern algorithms can be seen to have better performance, where this refers to the following:
1. Bounding Box Precision - it provides the correct bounding box where it is not too large or narrow.
2. Recall - it is able to find all the objects.
3. Class Prediction - it is able to output the correct class/label for each of the found object.
4. Speed of the algorithm - it is where the models are getting faster and faster at computing the results so that it can be used in real time. (real time = 5fps for computer vision tasks.)

## 2 - Evaluating the Object Detection model's Performance:

Before diving into YOLO's architecture or going further, it is important to cover some basics in model evaluation that are related to the YOLO model. Evaluating an object detection model will require more than accuracy of the prediction against the ground truth, these extra metrics to be included here are called __Precision__ and __Recall__. These will serve as the basis to compute other metrics that are important to object detection. 

## 2.1 - Precision and Recall Metrics:

To begin with, here are the formulas for these metrics:

$$ precision = \frac{TP}{TP - FP} $$

and

$$ recall = \frac{TP}{TP - FN} $$

where here,

- TP = Number of True Positives (like how many prediction are matching the ground truth of the same class) 
- FP = Number of False Positives (like how many prediction that do not match the ground truth for the same class)
- FN = Number of False Negatives (like how many ground truths do not have a matching prediction)

Additionally, which is not included in the formulas above, but are used in other areas/topics:
- TN = Number of True Negatives (like how many ground truths are actualy a negative example)

If the predictions are exactly matching the ground truthss, both FP and FN won't exist. In this case following the formulas, both precision and recall would equal to 1, leading to a perfect score. 

When a model's predictions comes from non-robust features during its training, the precision of the overall model will decrease because there will be a rise in false positives. In contrast, if the model is too strict where it detects only under precise conditions are met, the recall will decrease due to a rise in false negatives.

## 2.2 - Precision and Recall Curve:

Precision-Recall curves are used to summarise the trade off between the TP rate and the posistive predictive calue for a model by using different probability thresholds. The ideal model should have both high precision and high recall, however, most models often have a trade off between the two. One way to tell how well the model is performing is to compare the area under the curve (AUC) of the chart, where if the area is larger then it performs better or is a better classifier. Below shows a chart that compares two models where the BLUE line model is a better model compared to the GREEN line model. 

Source: 
- https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
- https://www.geeksforgeeks.org/precision-recall-curve-ml/

<img src="Description Images/Precision Recall Curve.png" width="350">

Image Ref -> https://www.geeksforgeeks.org/precision-recall-curve-ml/

Since the general idea is to visualise the performance (precision and recall) at each threshold of confidence, where for every output bounding box, the model will also output a confidence (between 0 and 1, where 1 is more confident). In practice it is a good idea to remove less confident predictions and this is done by setting a certain threshold (T = 0.48 or 0.4) so that any predictions below this number will be removed. 

Changing the value of the threshold will impact the model's precision and recall, where for:
1. __If T = close to 1__: The Precision will be high and the Recall will be low. This means that many objects will be filtered out and the model will miss it as only the confident predictions are kept.
2. __If T = close to 0__: The Recall will be high and the Precision will be low. This means that most of the predictions will be kept and there will not be any false negatives. This directly contributes to the rise of false positive results where the model is less confident in its predictions. 

The choice of this threshold is also high reliant on the problem at hand (classification task). For example, for a model to detect pedestrians, the ideal model would have a high recall rate so not to misss any passers-by, even if the car have to stop for no reason. For a model to detect investment opportunities, the ideal model will have a high precision rate to make sure that it doesn't miss good opportunities and avoids the wrong opportunities.

## 2.3 - Average Precision and Mean Average Precision:

For convenience in practice, often a single number to describe what is happening is very useful. In these cases there are
two terms to consider using: 

1. __Average Precision (AP)__: is the area under th curve and it is always between 0 and 1. This was meantioned earlier. It gives a good indicator of the model performance for a __Single__ class.
2. __Mean Average Precision (mAP)__: is the mean of the average precision for __each__ of the classes. It provides an overall __global scor__. 

## 2.4 - Average Precision Threshold: Jaccard Index (or Intersection over Union, IoU)

The __Jaccard Index__ or __Intersection over Union (IoU)__ is the common metric that is used to measure when a prediction and ground truth are matching. This also defines the TP and FP metrics earlier. The following equation defines the IoU a:

$$ IoU(A, B) =  \frac{\lvert{A \cap B}\rvert}{\lvert{A \cup B}\rvert} = \frac {\lvert{A \cap B}\rvert}{\lvert{A}\rvert - \lvert{B}\rvert - \lvert{A \cap B}\rvert} $$

Where here,
- $\lvert{A}\rvert$, is the cardinality of set A. The number of elements A contains.
- $\lvert{B}\rvert$, is the cardinality of set B. The number of elements B contains.
- $\lvert{A \cap B}\rvert$, is the numerator for number of elements that are in common between A and B. Here $A \cap B$ is the intersection between the two sets.
- $\lvert{A \cup B}\rvert$, is the demoninator for total number of elements that A and B sets covers. Here $A \cup B$ is the union of the sets.

#### The following diagram shows Precision, Recall and IoU:

<img src="Description Images/IoU intersection.PNG" width="600">

Image Ref -> http://www.gabormelli.com/RKB/Bounding_Box_Intersection_over_Union_(IoU)_Measure

Generally, the IoU are used in the computation rather than just the intersections, because the intersections value is absolute but not relative, meaning that two big boxes have more overlapping pixels than two smaller boxes, so the ratio is used instead as it is relative. IoU will always be between 0 and , where 0 if the two boxes does not overlap and 1 if the boxes overlaps completely.

## 3 - YOLO Detection Algorithm: 

YOLO which stands for You Only Look Once algorithm, is one of the fasted object detection algorithm that are available. One a decent GPU, it will be able to achieve >100 FPS (frames per second) for an image sized at 256 by 256 pixels. YOLO is not only fast but accurate too, it has several iterations over the last few years from 2015 to 2018, where the current version is v3. 

The main creator for the YOLO paper maintains the network framework (Called Darknet) here: https://github.com/pjreddie/darknet.

## 3.1 - Advantages and Disadvantages of YOLO:

As mentioned before, the YOLO model is praised for its speed, however, recently this model was outperformed by Fast R-CNN because it performs better than YOLO on detection of smaller objects. Another disadvantage is that the model struggles to classify objects that deviate from the original training dataset. 

## 3.2 - Main Concepts of YOLO:

From their paper (link below), they tackled the problem differently by "reframing object detection as a single regression problem". This essentially means that rather than using a sliding window or other techniques, the input image is divided into a width x height grid. This can be seen in the diagram below:

<img src="Description Images/YOLO Grid.PNG" width="350">

Image Ref -> https://www.analyticsvidhya.com/blog/2018/12/practical-guide-object-detection-yolo-framewor-python/

As it can be seen in each part of the grid, the bounding boxes are defined. This is then followed by predicting the (i) entre of the box, (ii) the width and height of the box, (iii) probability that there is an object within the box, and (iv) lastly the class of the object itself. It is because these predictions are computed as numbers, the problem can be transformed into a regression problem as stated. This covers roughly the basics of the YOLO's concept in detecting objects, there are also more complex issues such as, more than one object in the grid, overlapping objects in several parts of the grid, or how to train the loss for this model? 

Source: 
- https://arxiv.org/pdf/1506.02640v5.pdf
- https://towardsdatascience.com/yolo-you-only-look-once-real-time-object-detection-explained-492dc9230006

For the purpose of simplicity the model will be broken down into two parts, begining with the __Inference__ and then the __Training__ later on.

## 3.3 - YOLO Architecture: Part 1 Inference.

The YOLO architecture itself is based on a __Backbone model__ like most other CNNs, where the backbone model is the feature extractor that will extract meaningful features from the input images that will be used by the final layers later on. The feature extractor architecture here can be anything, but the official YOLO feature extractor uses its own custom backbone. Changing or altering the feature extractor can lead to different performance numbers. Below is a diagram that shows YOLO's model architecture:

<img src="Description Images/YOLO Architecture.PNG" width="850">

Image Ref -> http://datahacker.rs/how-to-peform-yolo-object-detection-using-keras/

The feature extractor's final layer will output a feature volume of size width x height x depth (w x h x D), where the width x height is the size of the grid and D is the depth of the feature volume. The size of the grid (w x h) will depend on the following two factors:

- Stride of the complete feature extractor.
- Size of the input image (smaller the input image, the smaller the grid).

The final layers of the model will take the feature volume as its input. The final layers are composed of 1x1 convolutional filters, which changes the depth of the feature volume without affecting its spatial structure.

## 3.4 - Outputs:

The final outputs from the model is sized as width x height x M (or w x h x M). The (w x h) is the grid size and $M = B * (C + 5)$. Where here, B is the number of bounding boxes per grid cell, C is the number of classes. Notice that there is a number 5 added to the number of classes. This is because, for each of the bounding boxes there is a need to predict (C + 5) of the following numbers:

1. $t_{x}$ and $t_{y}$ - is to compute the coordinates of the centre of the bounding box.
2. $t_{w}$ and $t_{h}$ - is to compute the width and height of the bounding box.
3. c - is the confidence of the object in the box.
4. $p1, p2, ..., $ and $pC$ - are  the probability that the bounding box has the object of class 1, 2, ..., C (where C = 20 for the  dataset classes).

Alternatively, $M = B * (C + 5)$ can be broken down further as $M = B * (C + 4 + 1)$. Where here, B is the number of bounding boxes per grid cell, C is the number of classes, 4 is for the bounding boxes and 1 is the objectness prediction.

Below the diagram summarises the output matrix:

<img src="Description Images/YOLO final matrix output.PNG" width="850">

Image Ref -> http://datahacker.rs/deep-learning-bounding-boxes/ and https://www.cnblogs.com/ranjiewen/p/7896398.html

## 3.5 - Anchor Boxes:

Prior to computing the final bounding boxes as mentioned above, the YOLO architecture also employs the concept of __Anchor Boxes__. This technique was because in version 1 of YOLO, many errors (because the objects varried in sizes) cropped up by directly computing the bounding box coordinates: $t_{x}, t_{y}, t_{w}, t_{h}$. Most of the objects in the training set were to big which resulted in larger bounding boxes, and the model would fail to draw these boxes around smaller objects. YOLO version 2 utilised Anchor Boxes to fix this issue.

__Anchor Boxes (a.k.a Priors) are a set of bounding boxes__ that were decided before training the network, and consists of 3 to 25 different sizes. The network will then refine these anchor boxes to find the closest matching ones for the object in the image. For example, in the image below, a set of 5 bounding boxes were picked to detect the car. As these boxes do not exactly match the object, the network will be required to refine the closest matching anchor box. 

<img src="Description Images/YOLO Anchor Box Example 1.PNG" width="350">

Image Ref -> https://www.jeremyjordan.me/object-detection-one-stage/

In this example, the network will need to correct the height and a small portion of the width of the anchor box. These are what the coordinates, $t_{x}, t_{y}, t_{w}, t_{h}$, previously mentioned above, they are the __corrections to the anchor box__ on the object. For YOLO version 2, the creators realised that sizes of anchor boxes are different for each dataset, therefore it was recommended that the data is to be analysed first to pick the size of the anchor boxes.

## 3.6 - Refining the Anchor Boxes:

The refinement process of these anchor boxes in practice are computed with the following equations:

$$ b_{x} = sigmoid(t_{x} + c_{x}) $$
$$ b_{y} = sigmoid(t_{y} + c_{y}) $$

$$ b_{w} = p_{w}exp(t_{w}) $$
$$ b_{h} = p_{h}exp(t_{h}) $$

Where here,
- $t_{x}, t_{y}, t_{w}, t_{h}$ are the outputs from the last layer.
- $b_{x}, b_{y}, b_{w}, b_{h}$ are the position and size of the predicted bounding box.
- $p_{w}$ and $p_{h}$ are the original size of the anchor box.
- $c_{x}$ and $p_{y}$ are the coordinates of the current grid cell, like (0, 0) for the top left box, (w-1, ) for the top right box, and (0, h-1) for the bottom left box.
- $exp$ is the exponential function.
- $sigmoid$ is the sigmoid function.

Below is a diagram to show how these equations are use:

<img src="Description Images/YOLO Anchor Boxes refinement.PNG" width= "400">

Image Ref -> https://arxiv.org/pdf/1612.08242v1.pdf

The output matrix of the neural network would need to be transformed into a list of boudning boxes, it also needs to be ran for every inference for the computation of the bounding boxes for an image, and below is a simplified version of the code:

In [9]:
# Converting the numbers of an output matrix from Neural Network to a list of bounding boxes:
if dont_run == 1:
    
    boxes = []
    for row in range(grid_height):
        for col in range(grid_width):
            for b in range(nb_box):
                tx, ty, tw, th = network_output[row, col, b, :4]
                box_confidence = network_output[row, col, b, 4]
                class_scores = network_output[row, col, b, 5:]
                
                # Bounding Boxes
                bx = signmoid(tx) + col
                by = sigmoid(ty) + row
                
                # Anchor boxes are a list of dict consisting of the size of each anchor:
                bw = anchor_boxes[b]['w'] * np.exp(tw)
                bh = anchor_boxes[b]['h'] * np.exp(th)
                
                boxes.append( (bx, by, bw, bh, box_confidence, class_scores))
    

## 3.7 - Post-processing the Bounding Boxes:

Before the boxes can be displayed, it will require a futher post-processing. From the above, the list consists of the cordinates, size of the predicted boxes, confidence and the class probabilities. However, right now, the class probabilities also includes predictions that are less confident or wrong, so these will need to be excluded. Utilising these, the confidence and the class probabilities are then multiplied firstly, then a threshold is used to include only those that are of greater probabilities.

Below is the code to show this example:

In [10]:
if dont_run == 1:
    
    # The confidence is a float variable, Classes are an array of the size equivalent to the nb_classes:
    final_scores = box_confidence * class_scores
    
    # Set the Threshold:
    OBJECT_THRESHOLD = 0.3
    
    # Filter out the lower probabilities and keep the ones that are greater than the threshold:
    filter_classes = class_scores >= OBJECT_THRESHOLD
    
    filtered_scores = class_scores * filter_classes
    
    # If the filtered scores contains non-null values, it means that at least one class is above the threshold and this is kept:
    class_id = np.argmax(filtered_scores)
    class_label = CLASS_LABELS[class_id]

After applying the filtering/post-processing to all of the bounding boxes in the grid, all the information can be used to draw the predictions. The following shows the output from this stag:

<img src="Description Images/YOLO ouput bounding boxes before NMS.png" width="350">

Image Ref -> https://towardsdatascience.com/guide-to-car-detection-using-yolo-48caac8e4ded

As it can be seen, there are numerous bounding boxes that are overlapping the objects in the grid. This is an outcome of the object occupying multiple grid boxes and therefore will have multiple bounding boxes overlapping each other. In order to correct this and clean up the image, the last stage of the post-processing pipeline is the __Non-Maximum Supression (NMS)__.

## 3.8 - Non-Maximum Supression (NMS):

The NMS post-processing procedure is to remove all the other unwanted bounding boxes (with the lowest probability) and keep only the bounding box with the highest probability. This is why it is call __Non-Maximum__ as it removes the boxes with lower probabilities. This can be done by firstly sorting out all the boxes by its probability, then taking the boxes with the highest probability. Next, for each of the box chosen, proceed to compute the __Intersection over Union (IoU)__ with all the other boxes.

Once the IoU(s) are computed between a box and the other boxes, the ones with an IoU value that is above a certain threshold (typically, 0.5 to 0.9) will be removed.

The following shows an example code for the NMS:

NOTE: In practice, TensorFlow has its own implementation of NMS called "tf.image.non_max_supression(boxes, ...)"

In [11]:
if dont_run == 1:
    
    # Sort the bounding boxes:
    sorted_boxes = sort_boxes_by_confidence(boxes)
    ids_to_suppress = []
    
    # Compute the IoU:
    for maximum_box in sorted_boxes:
        for idx, box, in enumerate(boxes):
            iou = compute_iou(maximum_box, box)
            
            if iou > iou_threshold:
                ids_to_suppress.append(idx)
                
    # Delete the unwanted boxes:
    processed_boxes = np.delete(boxes, ids_to_suppress)

After post-processing with NMS, the object now has only one bounding box surrounding it. This can be seen in the image below:

<img src="Description Images/YOLO ouput bounding boxes after NMS.png" width="600">

Image Ref -> https://towardsdatascience.com/guide-to-car-detection-using-yolo-48caac8e4ded

## 3.9 - Summary of YOLO Inference:

Before diving into YOLO training phase, this section will summarise all the steps taken. 

1. Input image enters the CNN backbone to compute the feature volumes.
2. Using the convolutional layer to compute the anchor box corrections, objectness scores, and class probabilities.
3. Using the output matrix to compute the coordinates of the bounding boxes.
4. Filter out the lower threshold boxes and post-process them with NMS to obtain the highest probability bounding box over the object.

It should be noted that this whole process is composed of convolutions and filtering operations throughout, which means that the model can ingest images of any size and ratios, giving the property of flexibility.

## 4 - YOLO Architecture: Part 2 YOLO Training.

The YOLO model is composed of two parts, where the first is the backbone and the second is the YOLO head. As for the backbone architectures, many types can be used. Typically before training the full YOLO model, the backbone would be trained first, on a classsification task with pretrained weights of ImageNet as this speeds up the computation time. 

Keras can be employed to train the backbone based on the pretrained wweights of ImageNet, the following shows an example:

In [12]:
if dont_run == 1:
    
    # Define the input image shape:
    input_image = Input(shape = (IMAGE_H, IMAGE_W, 3))
    
    # Define the True Box for the objects:
    true_boxes = Input(shape = (1, 1, 1, TRUE_BOX_BUFFER, 4))
    
    # Instantiate the Backbone CNN model: Inception model v.
    inception = InceptionV3(input_shape = (IMAGE_H, IMAGE_W, 3),
                            weights = 'imagenet',
                            include_top = False)
    
    # Get the output feature volumes from the inception model:
    features = inception(input_image)
    
    # Get the out shape:
    GRID_H, GRID_W = inception.get_output_shape_at(-1)[1:3]
    
    # Get the output with the bounding box:
    output = Conv2D(BOX * (4 + 1 + CLASS), 
                    (1, 1),
                    strides = (1, 1),
                    padding = 'same',
                    name = 'DetectionLayer',
                    kernel_initializer = 'lecun_normal')(features)
    
    # Reshape the output:
    output = Reshape( (GRID_H, GRID_W, BOX, 4 + 1 + CLASS) )(output)

## 4.1 - YOLO Loss:

Immediately it can be seen that the output of the last layer is unusual compared to model that were implemented in previous notebooks, therefore the corresponding loss here will also be unusual. Note that the YOLO loss is quite complex and will be required to be broken down into several parts. Each of these corresponding loss outputs will be returned by the last layer.

The YOLO network will predict the following kinds of information:
- Bounding box coordinates and its size. (__Bounding Box Loss__)
- Cofidence that the object of interest is in the bounding box. (__Object Confidence Loss__)
- Scores of the classes. (__Classification Loss__)
- Full YOLO Loss.

In general, the loss computed here should be high when the error is high and the loss will be used to penalise the incorrect values. However, with YOLO, the loss should only penalise when it makes sense to do so, such as if the bounding box contains no objects, there is no need to penalise its coordinates as it won't be utilised anyway. 

NOTE: As the source paper does not include how the loss is computed, it is better to check other sources of their loss implementation as there are several that works.

### 4.1.1 - Boudning Box Loss:

This first loss aids the model to learn the weights so that it can be used to predict the bounding box coordinates and its size. Key to this loss equation would be the __indicator function__, where the computed coordinates will only be correct when the box is responsible in detecting an object. The key part of this loss computation is the indicator function, where the coordinates will be correct if the box is responsible for detecting an object. Taking the instance of YOLO v1, it had difficulty in determining the correct bounding box to use, and in YOLO v2, it was resolved by using the highest IoU with the detected object. The goal here is to ensure that each objectof interest will have its own anchor boxes.

The following equation shows the loss at this stage:

$$ \lambda_{coord} \sum^{S^{2}}_{i=0} \sum^{B}_{j=0} 1_{ij}^{obj} [(x_{i} -\hat{x_{i}})^{2}] + \lambda_{coord} \sum^{S^{2}}_{i=0} \sum^{B}_{j=0} 1_{ij}^{obj} [(\sqrt{w_{i}} - \sqrt{\hat{w_{i}}})^{2} + (\sqrt{h_{i}} - \sqrt{\hat {h_{i}}})^{2}] $$

Where here,
- $ \lambda_{coord} $ is the weighting of the loss, it determines how much importance is given to the bounding box coordinates during training.
- $ \sum^{S^{2}}_{i=0} \sum^{B}_{j=0} $ is the sum for each part of the grid (from i = 0 to i = $S^{2}$) and for each box in this part of the grid (from 0 to B).
- $ 1_{ij}^{obj} $ is the indicator function for objects, it is equal to 1 when the $i^{th}$ part of the grid and the $j^{th}$ bounding box are responsible for an object.
- $ x_{i}, y_{i}, w_{i}m h_{i}$ is the bounding box size and coordinates. The difference between the predicted value (hat^) from the ouput of the network and the target value (ground truth). 
- $ Square Root $ ensures that the values are positive.
- By taking the square root of $w_{i}$ and $h_{i}$, it ensures that the errors for small bounding boxes are penalised more heavily compared to larger bounding boxes.

### 4.1.2 - Object Confidence Loss:

The next loss computation will have the network learn the weights that predicts if the bounding box contains an object.

The following equation shows the loss equation:

$$ \lambda_{obj} \sum^{S^{2}}_{i=0} \sum^{B}_{j=0} 1_{ij}^{obj} [(C_{ij} -\hat{C_{ij}})^{2}] + \lambda_{noobj} \sum^{S^{2}}_{i=0} \sum^{B}_{j=0} 1_{ij}^{noobj} [(C_{ij} -\hat{C_{ij}})^{2}] $$

Where here,
- $C_{ij}$ is the confidence that the box (j) in the part (i) of the grid does contain an object.
- $1_{ij}^{noobj}$ is the indicator function for no object, it will equal to 1 when the $i^{th}$ part of the gird and the $j^{th}$ bounding box are not responsible for an object.

In order to ensure that the objectness score is not penalised when there are other good candidates that also fits the object of interest, the $1_{ij}^{noobj}$ can be defined as seen below:

$$ 1_{ij}^{noobj} == \{^{1 (box is not responsible for any object) and (box not overlapping too much with any object bounding box)}_{0 (otherwise)}$$

Note that in practice, for each of the bounding box (i, j), the IoU w.r.t the ground truth boxes is computed. If the IoU is over the threshold (e.g. 0.6), $1_{ij}^{noobj}$ would be set to 0. This ensures to avoid penalising the boxes that contain objects but are not responsible for the object of interest.

### 4.1.3 - Classification Loss:

The last part of the loss computation is the classification loss which have the network learning to predict the proper class for each of the bounding box. The following shows the loss equation:

$$ \sum^{S^{2}}_{i=0} 1_{i}^{obj} \sum_{c \epsilon classes} (p_{i}(c) - \hat{p_{i}}(c))^{2} $$

NOTE: the original YOLO paper utilised L2 loss while many other implementations were using cross-entropy.

### 4.1.4 - Full YOLO Loss:

With the above three loss equations stated, the final overall loss is computed by summing the three losses together. This means that the combination computes the loss penalises the error of the bounding box coordinate refinement, objectness scores and the class prediction. By backpropagating this error, the YOLO model can be trained properly to predict the correct bounding boxes for the correct objects.

## 5 - Techniques used for Training:

Using the loss, the YOLO model can be trained through backpropagation. To ensure that the loss does not diverge away and that good performance were to be obtained, techniques such as:

1. Augmentation, is used to prevent overfitting the dataset.
2. Multi-scaling training, where for every "n" batches, the network's input is changed to a different siz, which ensures the model learns to predict accurately across several input dimensions.
3. Pretraining the YOLO model on image classification task.
4. Burn-in, where the learning rate is reduced at the beginning of the training process to avoid a loss explosion. 

## 6 - Further Reading:

It is recommended to have a go at these three YOLO papers:
1. https://arxiv.org/pdf/1506.02640.pdf
2. https://arxiv.org/pdf/1612.08242.pdf
3. https://pjreddie.com/media/files/papers/YOLOv3.pdf

## 7 - Building the YOLO model in TensorFlow:

This section will now go through the implemenetation of the YOLO v3 model in TensorFlow 2.0++. Before getting started, download and save the "yolov3.weights" from pjreddie.com by using the following command:

wget https://pjreddie.com/media/files/yolov3.weights

Note that if the above command does not work, you can manually go to the website and download it. Link: https://github.com/AlexeyAB/darknet

Instructions for manual download:

__Step 1__:

Navigate through the Github Readme file and find the following:

<img src="Description Images/Manual Download of YOLO weights_1.PNG" width="600">

Image Ref -> https://github.com/AlexeyAB/darknet

__Step 2__:

Expand and Download the weights file.

<img src="Description Images/Manual Download of YOLO weights_2.PNG" width="600">

Image Ref -> https://github.com/AlexeyAB/darknet

__Step 3__:

Create your own folder inyour working directory for the weights so that it can be found later.

### 7.1 - Import the required Libaries:

In [13]:
import cv2

from absl import logging
from itertools import repeat

from tensorflow.keras import Model
from tensorflow.keras.layers import (Add, Concatenate, Lambda, 
                                     Conv2D, Input, LeakyReLU, 
                                     MaxPool2D, UpSampling2D, ZeroPadding2D)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import (binary_crossentropy, 
                                     sparse_categorical_crossentropy)

### 7.2 - Define the Global Variables:

Note that, the paths to the weights file (.weights) and its checkpoints (.tf) file will be placed in a .py utility file as well. 

In [14]:
# Define the Threshold for YOLO:
yolo_iou_threshold = 0.6

# Define the Threshold Score:
yolo_score_threshold = 0.6

# Define the path directory to the weigths file:
weightsyolov3 = 'yolov3.weights'

# Define the path directory to the checkpoints file:
weights = 'checkpoints/yolov3.tf'

# Resize the image size:
img_size = 416

# Define the checkpoint file:
checkpoints = 'checkpoints/yolov3.tf'

# Define the number of classes in the model:
nb_classes = 80


### 7.3 - List the Layers of the Yolo v3 model's Fully Convolutional Network:

In [15]:
YOLOV3_LAYER_LIST = ['yolo_darknet', 
                     'yolo_conv_0',
                     'yolo_output_0',
                     'yolo_conv_1',
                     'yolo_output_1',
                     'yolo_conv_2',
                     'yolo_output_2',]

### 7.4 - Load in the pretrained weights:

The YOLO v3 implementation utilised here is a fully convolutional network and is done without any training, but in order to predict an output with the model, pretrained weights are needed. This was the reason the weights were downloaded and saved locally first. Therefore, the pretrained weights will need to be loaded in and to do this, a function is need.

To aid the model in speeding up its computations, batch normalisation will be employed to normalise the outputs from the layers. Note that "tf.keras.layers.BatchNormalization" would not work here as this whole implementation utilises Transfer Learning. The fix is also included in the code below.


In [16]:
def load_darknet_weights(model, weights_file):
    wf = open(weights_file, 'rb')
    major, minor, revision, seen, _ = np.fromfile(wf, dtype = np.int32, count = 5)
    layers = YOLOV3_LAYER_LIST
    
    for layer_name in layers:
        sub_model = model.get_layer(layer_name)
        
        for i, layer in enumerate(sub_model.layers):
        
            # If the layer name does not start with 'conv2d', skip it and move to the next layer:
            # As this is the convolutional layer, there is no batch_norm set.
            if not layer.name.startswith('conv2d'):
                continue
            batch_norm = None
            
            # For all the layers that consist of Batch Norm within the model, if the layer starts with "batch_norm"
            # set it as the batch_norm variable:
            if i + 1 < len(sub_model.layers) and sub_model.layers[i + 1].name.startswith('batch_norm'):
                batch_norm = sub_model.layers[i + 1]
            
            # Log the layers layout for each sub_model:
            logging.ifo("{}/{} {}".format(sub_model.name, layer.name, 'bn' if batch_norm else 'bias'))
            
            # Define the filters:
            filters = layer.filters
        
            # Define the filter size:
            size = layer.kernel_size[0]
            
            # Define the input dimensions:
            input_dim = layer.input_shape[-1]
            
            # When batch_norm is None, set it to convolutional bias:
            if batch_norm is None:
                conv_bias = np.fromfile(wf, dtype = np.float32, count = filters)
            else:
                # darknet [beta, gamma, mean, variance]:
                bn_weights = np.fromfile(wf, dtype=np.float32, count = 4 * filters)
                
                # tf [gamma, beta, mean, variance]:
                bn_weights = bn_weights.reshape( (4, filters) )[[1, 0, 2, 3]]
            
            # darknet shape (out_dim, in_dim, height, width):
            conv_shape = (filters, input_dim, size, size)
            
            conv_weights = np.fromfile(wf, dtype = np.float32, count = np.product(conv_shape))
            
            # tf shape (height, width, in_dim, out_dim):
            conv_weights = conv_weights.reshape(conv_shape).transpose( [2, 3, 1, 0] )
            
            # Set the weights for the Convolutional layers and the bias:
            if batch_norm is None:
                layer.set_weights( [conv_weights, conv_bias] )
            else:
                # Set the weights for the batch norm:
                layer.set_weights( [conv_weights] )
                batch_norm.set_weights(bn_weights)
                
    assert len(wf.read()) == 0, 'failed to read the weights.'
    
    wf.close()   
    

In [17]:
class BatchNormalisation(tf.keras.layers.BatchNormalization):
    """ This builds the Batch Normalisation function for YOLOv3 to be compatible with Transfer Learning Methods.
        It also inherits the properties from tf.keras.layers.BatchNormalization.
    
    Parameters: within the "call" method.
        - Make trainable = False freeze BN for real.
    """
    def call(self, x, training = False):
        if training is None:
            training = tf.constant(False)
            
        training = tf.logical_and(training, self.trainable)
            
        return super().call(x, training)

### 7.5 - Define the function computing the Intersection over Union (IoU):

The following two functions will compute the "intersection" and the "IuO" as seen in the diagram below:

<img src="Description Images/IoU intersection_2.PNG" width="350">

Image Ref -> http://www.gabormelli.com/RKB/Bounding_Box_Intersection_over_Union_(IoU)_Measure

In [18]:
def interval_overlap(interval_1, interval_2):
    x1, x2 = interval_1
    x3, x4 = interval_2
    
    if x3 < x1:
        return 0 if x4 < x1 else (min(x2, x4) - x1)
    else:
        return 0 if x2 < x3 else (mi(x2, x4) - x3)
    

In [19]:
def intersectionOverUnion(box1, box2):
    # Compute the intersect width and height:
    intersect_width = interval_overlap( [box1.xmin, box1.xmax], [box2.xmin, box2.xmax] )
    intersect_height = interval_overlap( [box1.ymin, box1.ymax], [box2.ymin, box2.ymax] )
    
    # Compute the intersection area:
    intersect_area = intersect_width * intersect_height
    
    # Get the length of height and width for both boxes:
    w1 = box1.xmax - box1.xmin
    h1 = box1.ymax - box1.ymin
    
    w2 = box2.xmax - box2.xmin
    h2 = box2.ymax - box2.ymin
    
    # Compute the Area of Union:
    union_area = (w1 * h1) + (w2 * h2) - intersect_area
    
    return float(intersect_area) / union_area

### 7.6 - Anchor Boxes:

For each of the scales (13x13 scale, 26x26 scale and 52x52 scale), the 3 anchor boxes will be defined for each grid.

The mask are:
- 0, 1, 2 are the masks to use for the first three anchor boxes.
- 3, 4, 5 are the masks to use for the fourth, fifth and sixth boxes.
- 6, 7, 8 are the masks to use for the seventh, eighth and ninth boxes.

In [20]:
# Define the Yolo Anchor boxes:
yolo_anchors = np.array( [(10, 13), (16, 30), (33, 23), 
                          (30, 61), (62, 45), (59, 119), 
                          (116, 90), (156, 198), (373, 326)], np.float32 ) / 416

# Define the Yolo Anchor Masks:
yolo_anchor_masks = np.array( [[6, 7, 8], [3, 4, 5], [0, 1, 2]] )


### 7.7 - Function to draw the Bounding Boxes, Class Name and Probability:

In [21]:
def draw_outputs(img, outputs, class_names):
    boxes, score, classes, nums = outputs
    boxes, score, classes, nums = boxes[0], score[0], classes[0], nums[0]
    
    wh = np.flip(img.shape[0:2])
    
    for i in range(nums):
        x1y1 = tuple( (np.array(boxes[i][0:2]) *  wh).astype(np.int32) )
        x2y2 = tuple( (np.array(boxes[i][2:4]) *  wh).astype(np.int32) )
        
        img = cv2.rectangle(img, x1y1, x2y2, (255, 0, 0), 2)
        
        img = cv2.putText(img = img, 
                          text = '{} {:.3f}'.format(class_names[int(classes[i])], score[i]), 
                          org = x1y1, 
                          fontFace = cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                          fontScale = 1,
                          color = (0, 0, 255), 
                          thickness = 2)
        
        return img

### 7.8 - Implement YOLOv3 Network Architecture:

As seen in the diagram below, the Fully Convolutional layers consists of 53 layers. The Functional API would be used to implement these layers effectively and these layers can be altered by simply passing the stated overall parameters into the model functions. It also means that the branches such as the Residual Block can be defined as well as easily implemented amongst the layers. The model architecture uses Residual Blocks for learning the features of the input images. 

<img src="Description Images/YOLO Architecture_2.PNG" width="700">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

Below shows the Residual Block:

<img src="Description Images/Residual Block.PNG" width="500">

Image Ref -> https://towardsdatascience.com/architecture-comparison-of-alexnet-vggnet-resnet-inception-densenet-beb8b116866d

#### For DarknetConv block:

The code design follows accordingly:

<img src="Description Images/DarknetConv.PNG" width="400">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [22]:
def DarknetConv(x, filters, size, strides = 1, batch_norm = True):
    """ This builds the Convolutions Filters over the input image.
    Parameters:
        - x, is the input image tensor.
        - filters, is the number of filters/kernels.
        - size, is the size of the kernels.
        - strides, is the convoluitional stride.
        - batch_norm, is a Flas to use or not the batch normalisation in this convolutional layer.
    Returns:
        - returns the processed (convolved) x output.
    """
    # Set the padding to ouput the same dimension/shape as the input if stride = 1:
    if strides == 1:
        padding = 'same'
    else:
        # Padding applied to only the top left of the input tensor:
        x = ZeroPadding2D( padding=((1, 0), (1, 0)) )(x)
        x = Conv2D(filters = filters,
                   kernel_size = size,
                   strides = strides,
                   padding= padding,
                   use_bias = not batch_norm,
                   kernel_regularizer = l2(l = 0.0005)
                  )(x)
        
    # If this layer uses batch_norm, set the following:
    if batch_norm:
        x = BatchNormalisation()(x)
        x = LeakyReLU(alpha=0.1)(x)

    return x

#### For DarknetResidual block:

The code desgin follows accordingly:

<img src="Description Images/DarknetResidual.PNG" width="400">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [23]:
def DarknetResidual(x, filters):
    """ This builds the Residual block.
    Parameters:
        - x, is the input tensor.
        - filters, is the number of filters/kernels
    Returns:
        - returns the processed x output.
    """
    # Save an instance of x to be used for merging later:
    previous_x = x
    
    # 1st DarknetConv_BatchNorm_LeakyReLU (DBL) block:
    x = DarknetConv(x = x,
                    filters = filters // 2,
                    size = 1)
    # 2nd DarknetConv_BatchNorm_LeakyReLU (DBL) block:
    x = DarknetConv(x = x,
                    filters = filters,
                    size = 3)
    
    # Merge the Residual and DBL:
    x = Add()( [previous_x, x] )
    
    return x

#### For the Residual Body Block:

The code desgin follows accordingly:

<img src="Description Images/DarknetResidual_Body.PNG" width="400">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [24]:
def DarknetBlock(x , filters, blocks):
    """ This builds the Resblock_body block.
    Parameters:
        - x, is the input tensor.
        - filters, is the number of kernels/filters.
        - blocks, is the number of repeated blocks to be set.
    Returns:
        - returns the processed x output of this ResN block.
    """
    # 1st block with zero padding and DarknetConv_BatchNorm_LeakyReLU (DBL) block:
    x = DarknetConv(x = x,
                    filters = filters,
                    size = 3, 
                    strides=2)
    
    # Repeated ResN blocks (or Redisdual blocks):
    for _ in repeat(None, blocks):
        x = DarknetResidual(x = x, 
                            filters = filters)   
    
    return x


#### For 53-Layer Darknet Fully Convolutional Network:

The code desgin follows accordingly:

<img src="Description Images/Darknet_53.PNG" width="400">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [25]:
def Darknet(name = None):
    """ This builds the 53-Layer Darknet model.
    Parameters:
        - name, is the Name Suffix of this layer.
    Returns:
        - returns the model.
    """
    # Input:
    x = inputs = Input( [None, None, 3] )
       
    # 1st layer after input: DarknetConv_BatchNorm_LeakyReLU (DBL) block.
    x = DarknetConv(x = x, filters = 32, size = 3)
        
    # 2nd Block: res1 block.
    x = DarknetBlock(x = x, filters = 64, blocks = 1)
    
    # 3rd Block: res2 block.
    x = DarknetBlock(x = x, filters = 128, blocks = 2)
    
    # 4th Block: res8 block.
    x = x_36 = DarknetBlock(x = x, filters = 256, blocks = 8)
    
    # 5th Block: res8 block.
    x = x_61 = DarknetBlock(x = x, filters = 512, blocks = 8)
    
    # 6th Block: res4 block.
    x = DarknetBlock(x = x, filters = 1024, blocks = 4)
    
    return tf.keras.Model(inputs, (x_36, x_61, x), name = name)

#### For Yolo Convolutions:

The code desgin follows accordingly:

<img src="Description Images/Yolo_conv.PNG" width="700">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [26]:
def YoloConv(filters, name = None):
    """ This builds the Yolo Convolutions (DBL * 5) and the (DBL + Up-Sampling).
    Parameters:
        - filters, is the number of filters.
        - name, is the Name Suffix of this layer.
    Returns:
        - returns a callable yolo_conv layer.
    """
    def yolo_conv(x_in):
        
        # For DBL with Up-sampling:
        if isinstance(x_in, tuple):
            inputs = Input(x_i[0].shape[1:]), Input(x_in[1].shape[1:])
            
            x, x_skip = inputs
            
            x = DarknetConv(x = x, filters = filters, size = 1)
            x = UpSampling2D(2)(x)
            x = Concatenate()( [x, x_skip] )
        
        else:
            x = inputs = Input(x_in.shape[1:])
            
        # For DBL * 5 Layer:
        x = DarknetConv(x = x, filters = filters, size = 1)
        x = DarknetConv(x = x, filters = filters * 2, size = 3)
        x = DarknetConv(x = x, filters = filters, size = 1)
        x = DarknetConv(x = x, filters = filters * 2, size = 3)
        x = DarknetConv(x = x, filters = filters, size = 1)
        
        return Model(inputs, x, name = name)(x_in)
    return yolo_conv          


#### For Yolo Outputs, Yolo Boxes and Non Maximum Suppression:

The code desgin follows accordingly:

<img src="Description Images/Yolo_output.PNG" width="700">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [27]:
def YoloOutput(filters, anchors, classes, name = None):
    """ This builds the Yolo Output blocks, where it can be configured for different channels.
    Parameters:
        - filters, is the number of filters/kernels.
        - anchors, is the defined number of anchor boxes (masks).
        - classes, is the number of classes.
        - name, is the Name Suffix of this layer.
    Returns:
        - returns a callable yolo output layer.
    """
    def yolo_output(x_in):
        x = inputs = Input( x_in.shape[1:] )
        x = DarknetConv(x = x, filters = filters * 2, size = 3)
        x = DarknetConv(x = x, filters = anchors * (classes + 5), size = 1, batch_norm = False)
        x = Lambda(lambda x: tf.reshape(tensor = x, 
                                        shape = (-1, tf.shape(x)[1], tf.shape(x)[2], anchors, classes + 5)
                                        )
                  )(x)
        
        return tf.keras.Model(inputs, x, name = name)(x_in)
    return yolo_output       
        

In [28]:
def yolo_boxes(pred, anchors, classes):
    """ This builds the YOLO model output prediction boxes.
    Parameters:
        - pred, is the input predictions from the model.
        - anchors, is the defined number of anchor boxes (masks).
        - classes, is the number of classes.
    Returns:
        - returns the bounding boxes (bbox), score, class_probs, and pred_box.
    """
    # Define the Grid Size:
    grid_size = tf.shape(pred)[1]
    
    # Define the box x-y, w-h, score and class_prob:
    box_xy, box_wh, score, class_probs = tf.split(pred, (2, 2, 1, classes), axis=-1)
    
    # Refine the Acnhor boxes (part 1):
    box_xy = tf.sigmoid(box_xy)
    score = tf.sigmoid(score)
    class_probs = tf.sigmoid(class_probs)
    pred_box = tf.concat( (box_xy, box_wh), axis = -1)
    
    # Build the Grid:
    grid = tf.meshgrid(tf.range(grid_size),
                       tf.range(grid_size))
    grid = tf.expand_dims(tf.stack(grid, axis = -1), 
                          axis = 2)
    
    # Refine the Anchor Boxes (part 2):
    box_xy = (box_xy + tf.cast(grid, tf.float32)) / tf.cast(grid_size, tf.float32)
    box_wh = tf.exp(box_wh) * anchors
    
    box_x1y1 = box_xy - box_wh / 2
    box_x2y2 = box_xy + box_wh / 2
    
    bbox = tf.concat( [box_x1y1, box_x2y2], axis = -1)
    
    return bbox, score, class_probs, pred_box  
    

In [29]:
def nonMaximumSuppression(outputs, anchors, masks, classes):
    """ This builds the NMS, where it remove all the other unwanted bounding boxes (with the lowest probability).
    Parameters:
        - outputs, is the output tensor from the previous layer.
        - anchors, is the number of anchors.
        - masks, is the anchor masks.
        - classes, is the number of classes.
    Returns:
        - returns boxes, scores, classes, valid_detections.
    """
    # Define empty arrays:
    boxes, conf, out_type = [], [], []
    
    # Append the empty arrays:
    for output in outputs:
        boxes.append(tf.reshape(tensor = output[0],
                                shape = (tf.shape(output[0])[0], -1, tf.shape(output[0])[-1])
                               )
                    )
        conf.append(tf.reshape(tensor = output[1],
                                shape = (tf.shape(output[1])[0], -1, tf.shape(output[1])[-1])
                               )
                    )
        out_type.append(tf.reshape(tensor = output[2],
                                   shape = (tf.shape(output[2])[0], -1, tf.shape(output[2])[-1])
                                  )
                       )
        
    # Define the bounding box, confidence and class_probs: axis=1 means concat to the right side, not below.
    bbox = tf.concat(values = boxes, axis = 1)
    confidence = tf.concat(values = conf, axis = 1)
    class_probs = tf.concat(values = out_type, axis = 1)
    
    # Compute the scores:
    scores = confidence * class_probs
    
    # Apply TensorFlow NMS:
    boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(boxes = tf.reshape(bbox, (tf.shape(bbox)[0], -1, 1, 4)),
                                                                                     scores = tf.reshape(scores, (tf.shape(scores)[0], -1, tf.shape(scores)[-1])),
                                                                                     max_output_size_per_class = 100,
                                                                                     max_total_size = 100,
                                                                                     iou_threshold = yolo_iou_threshold,
                                                                                     score_threshold = yolo_score_threshold
                                                                                    )
    
    return boxes, scores, classes, valid_detections
    
        

#### For Entire YOLOv3 Network model:

The code desgin follows accordingly:

<img src="Description Images/YOLO Architecture_2.PNG" width="600">

Image Ref -> http://datahacker.rs/tensorflow2-0-yolov3/

In [30]:
def YoloV3(size = None, channels = 3, anchors = yolo_anchors, masks = yolo_anchor_masks, classes = 80, training = False):
    """ This builds the YoloV3 model, by combining all the blocks.
    Parameters:
        - size, is the kernels/filers size.
        - channels, is the number of RGB =3 colour channels.
        - anchors, is the yolo anchors.
        - masks, is the yolo anchors masks.
        - classes, is the number of classes.
        - training, is a Flag to state whether the model instantiation will will to be trained or not.
    Returns:
        - returns the YoloV3 model.
    """
    # Input layer:
    x = inputs = Input( [size, size, channels] )
    
    # Instantiate the Darknet-53 without FC layer:
    x_36, x_61, x = Darknet(name = 'yolo_darknet')(x)
    
    # Define the yolo_head blocks for each type of (512, 256, 128) or (1024, 512, 256) chanel: 
    # For masks -> [6, 7, 8]
    x = YoloConv(filters = 512, nam = 'yolo_conv_0')(x)
    output_0 = YoloOutput(filters = 512, anchors = len(masks[0]), classes = classes, name = 'yolo_output_1')(x)
    
    # For masks -> [3, 4, 5]
    x = YoloConv(filters = 256, nam = 'yolo_conv1')(x, x_61)
    output_1 = YoloOutput(filters = 256, anchors = len(masks[1]), classes = classes, name = 'yolo_output_2')(x)
    
    # For masks -> [0, 1, 2]
    x = YoloConv(filters = 128, nam = 'yolo_conv_2')(x, x_36)
    output_2 = YoloOutput(filters = 128, anchors = len(masks[2]), classes = classes, name = 'yolo_output_2')(x)
    
    
    # If Flag 'training' = True:
    if training:
        return Model(inputs, (output_0, output_1, output_2), name = 'yolov3')
    
    # Compute the bounding boxes:
    boxes_0 = Lambda(lambda x: yolo_boxes(pred = x, 
                                          anchors = anchors[masks[0]], 
                                          classes = classes), name = 'yolo_boxes_0')(output_0)
    boxes_1 = Lambda(lambda x: yolo_boxes(pred = x, 
                                          anchors = anchors[masks[1]], 
                                          classes = classes), name = 'yolo_boxes_1')(output_1)
    boxes_2 = Lambda(lambda x: yolo_boxes(pred = x, 
                                          anchors = anchors[masks[2]], 
                                          classes = classes), name = 'yolo_boxes_2')(output_2)
    
    # Apply NMS on the candidate bounding boxes:
    outputs = Lambda(lambda x: nonMaximumSuppression(outputs = x, 
                                                     anchors = anchors, 
                                                     masks = masks, 
                                                     classes = classes), 
                     name = 'nonMaximumSuppression')( (boxes_0[:3], boxes_1[:3], boxes_2[:3]) )
    
    
    return MOdel(inputs, outputs, name = 'yolov3')

#### For the model loss computations:

In [31]:
def YoloLoss(anchors, classes = 80, ignore_threshold = 0.5):
    """ This builds for the compute of the model losses.
    Parameters:
        - anchors, is the yolo_anchors or anchor boxes.
        - classes, is the number of classes.
        - ignore_threshold, is when not specified, the threshold will be default at 0.5.
    Returns:
        - returns yolo_loss, where = xy_loss + wh_loss + obj_loss + class_loss.
    """
    def yolo_loss(y_true, y_pred):
        
        # Part 1 - Transform all pred outputs:
        # y_pred: (batch_size, grid, grid, anchors, (x, y, w, h, obj, ...cls))
        pred_box, pred_obj, pred_class, pred_xywh = yolo_boxes(pred = y_pred, anchors = anchors, classes = classes)
        
        # Split the predicted box shape lines:
        pred_xy = pred_xywh[..., 0:2]
        pred_wh = pred_xywh[..., 2:4]
        
        # Part 2 - Transform all true outputs
        # y_true: (batch_size, grid, grid, anchors, (x1, y1, x2, y2, obj, cls))
        true_box, true_obj, true_class_idx = tf.split(value = y_true, num_or_size_splits = (4, 1, 1), axis = -1)
        
        # Split the Ground truth box shape lines:
        true_xy = (true_box[..., 0:2] + true_box[..., 2:4]) / 2
        true_wh = true_box[..., 2:4] - true_box[..., 0:2]
        
        # Allow for higher weights to small boxes:
        box_loss_scale = 2 - true_wh[..., 0] * true_wh[..., 1]
        
        # Part 3 - Inverting the pred box equations:
        grid_size = tf.shape(y_true)[1]
        grid = tf.meshgrid(tf.range(grid_size, tf.range(grid_size)))
        grid = tf.expand_dims(tf.stack(grid, axis = -1), axis = 2)
        
        # Convert true_xy to match grid size:
        true_xy = true_xy * tf.cast(grid_size, tf.float32) - tf.cast(grid, tf.float32)
        true_wh = tf.math.log(true_wh, anchors)
        true_wh = tf.where(tf.math.is_inf(true_wh), tf.zeros_like(true_wh), true_wh)
        
        # Part 4 - Compute all masks:
        obj_mask = tf.squeeze(input = true_obj, axis = -1)
        
        # Ignore when IoU is over the threshold stated:
        true_box_flat = tf.boolean_mask(tensor = true_box, mask = tf.casst(obj_mask, tf.bool))
        
        # Find the Best IoU:
        best_iou = tf.reduce_max(intersectionOverUnion(box1 = pred_box, box2 = true_box_flat), axis = -1)
        
        # Ignore IoU:
        ignore_mask = tf.cast(best_iou < ignore_threshold, tf.float32)
        
        # Part 5 - Compute all losses:
        xy_loss = obj_mask * box_loss_scale * tf.reduce_sum(input_tensor = tf.square(true_xy - pred_xy), axis = -1)
        wh_loss = obj_mask * box_loss_scale * tf.reduce_sum(input_tensor = tf.square(true_wh - pred_wh), axis = -1)
        
        # Using binary_crossentropy for the object loss:
        obj_loss = binary_crossentropy(y_true = true_obj, y_pred = pred_obj)
        obj_loss = obj_mask * obj_loss + (1 - obj_mask) * ignore_mask * obj_loss
        
        # Using sparse_categorical_crossentropy instead for the classes loss:
        class_loss= obj_mask * sparse_categorical_crossentropy(y_true = true_class_idx, y_pred = pred_class)
        
        # Part 6 - Sum over (batch, gridx, gridy, anchors) => (batch, 1)
        xy_loss = tf.reduce_sum(xy_loss, axis = (1, 2, 3))
        wh_loss = tf.reduce_sum(wh_loss, axis = (1, 2, 3))
        obj_loss= tf.reduce_sum(obj_loss, axis = (1, 2, 3))
        class_loss = tf.reduce_sum(class_loss, axis = (1, 2, 3))
        
        return xy_loss + wh_loss + obj_loss + class_loss
    
    return yolo_loss

#### For Tansforming targets:

Here, the funtion will transform targets outputs tuple of shape:

( \
  [N, 13, 13, 3, 6],\
  [N, 26, 26, 3, 6],\
  [N, 52, 52, 3, 6] \
)

Where here,

- N is the number of labels in the batch.
- 6 is [x, y, w, h, obj, class] of the bounding boxes.

In [32]:
@tf.function
def transform_targets_for_output(y_true, grid_size, anchor_idxs, classes):
    """ This builds the function to transform targets outputs tuple of shape.
        (
            [N, 13, 13, 3, 6],
            [N, 26, 26, 3, 6],
            [N, 52, 52, 3, 6]
        )
        
    Parameters:
        - y_true, is the labels.
        - grid_size, is the size of the grid.
        - anchor_idxs, is the anchor indexes.
        - classes, is the number of classes.
    Returns:
        - returns an updated scattered tensor.
    
    """
    # For y_true, shape is (N, boxes, (x1, y1, x2, y2, class, best_anchor))
    N = tf.shape(y_true)[0]

    # For y_true_out, shape is (N, grid, grid, anchors, [x, y, w, h, obj, class])
    y_true_out = tf.zeros( (N, grid_size, grid_size, tf.shape(anchor_idxs)[0], 6) )
    anchor_idxs = tf.cast(anchor_idxs, tf.int32)

    # Define array for indexes and updates:
    indexes = tf.TensorArray(tf.int32, size = 1, dynamic_size = True )
    updates = tf.TensorArray(tf.float32, size = 1, dynamic_size = True)

    # Loop over labels to get anchor boxes:
    idx = 0
    for i in tf.range(N):
        for j in tf.range(tf.shape(y_true)[1]):
            if tf.equal(y_true[i][j][2], 0):
                continue
            anchor_eq = tf.euqal(anchor_idxs, tf.cast(y_true[i][j][5], tf.int32))
            
            if tf.reduce_any(anchor_eq):
                box = y_true[i][j][0:4]
                box_xy = (y_true[i][j][0:2] + y_true[i][j][2:4]) / 2
                
                anchor_idx = tf.cast(tf.where(anchor_eq), tf.int32)
                grid_xy = tf.cast(box_xy // (1 / grid_size), tf.int32)
                
                indexes = indexes.write(idx, [i, grid_xy[1], grid_xy[0], anchor_idx[0][0]])
                updates = updates.write(idx, [box[0], box[1], box[2], box[3], 1, y_true[i][j][4]])
                idx += 1
                
    return tf.tensor_scatter_nd_update(y_true_out, indexes.stack(), updates.stack())


In [34]:
def transform_targets(y_train, anchors, anchor_mask, classes):
    """ This transforms the Outputs -> [x, y, w, h, object, class].
    Parameters:
        - y_train, are the labels from the training set.
        - anchors, are the anchor boxes.
        - anchor_masks, are the anchor boxes masks.
        - classes, is the number of classses.
    Returns:
        - returns a tuple of the output -> [x, y, w, h, obj, class]
    """
    # Define the array for the outputs:
    outputs = []
    
    # Define the Grid size:
    grid_size = 13
    
    # Compute the anchor index for true boxes:
    anchors = ttf.cast(anchors, tf.float32)
    anchor_area = anchors[..., 0] * anchors[..., 1]
    
    box_wh = y_train[..., 2:4] - y_train[..., 0:2]
    box_wh = tf.tile(input = tf.expand_dims(input = box_wh, axis = -2),
                     multiples = (1, 1, tf.shape(anchors)[0], 1))
    
    box_area = box_wh[..., 0] * box_wh[..., 1]
    
    intersection = tf.minimum(x = box_wh[..., 0], y = anchors[..., 0]) * tf.minimum(x = box_wh[..., 1], y = anchors[..., 1])
    iou = intersection / (box_area + anchor_area - intersection)
    
    anchor_idx = tf.cast(tf.argmax(iou, axis = -1), tf.float32)
    anchor_idx = tf.expand_dims(input = anchor_idx, axis = -1)
    
    y_train = tf.concat( [y_train, anchor_idx], axis = -1 )
    
    for anchor_i in anchor_masks:
        outputs.append(transform_targets_for_output(y_true = y_train, 
                                                    grid_size = grid_size,
                                                    anchor_idxs = anchor_i, 
                                                    classes = classes))
        grid_size *= 2
    
    return tuple(outputs)

In [35]:
def preprocess_image(x_train, size):
    return (tf.image.resize(x_train, (size, size))) / 255

## 8 - Run the Yolo v3 model:

First, instantiate the Yolov3 model. 

### Conclusion:


Source: 
- https://www.analyticsvidhya.com/blog/2018/12/practical-guide-object-detection-yolo-framewor-python/
- https://manalelaidouni.github.io/manalelaidouni.github.io/Understanding%20YOLO%20and%20YOLOv2.html
- https://appsilon.com/object-detection-yolo-algorithm/
- http://datahacker.rs/how-to-peform-yolo-object-detection-using-keras/
- http://datahacker.rs/tensorflow2-0-yolov3/
- https://github.com/zzh8829/yolov3-tf2
- https://github.com/AlexeyAB/darknet

## Summary:

